### 데이터 처리와 증강 

In [23]:
import tqdm

from torchvision import transforms as T
from torch.utils.data.dataloader import DataLoader

import torch.optim as optim

In [25]:
transform = T.Compose([
    T.Resize(224),
    T.RandomCrop((224, 224), padding=4),
    T.RandomHorizontalFlip(p=0.5),
    T.ToTensor(),
    T.Normalize(mean=(0.49, 0.48, 0.44), std=(0.24, 0.24, 0.26))
])

In [26]:
from torchvision.datasets.cifar import CIFAR10

train_data = CIFAR10(
    root='./',
    download=True,
    train=True,
    transform=transform
)

test_data = CIFAR10(
    root='./',
    download=True,
    train=False,
    transform=transform
)

  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./cifar-10-python.tar.gz to ./
Files already downloaded and verified


In [27]:
train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=True)

### 사전학습모델 준비 

In [4]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [30]:
device

'cuda'

In [2]:
# 사전학습모델 임포트
from torchvision.models.vgg import vgg16

model = vgg16(pretrained=True)

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


  0%|          | 0.00/528M [00:00<?, ?B/s]

In [5]:
model.classifier

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace=True)
  (2): Dropout(p=0.5, inplace=False)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace=True)
  (5): Dropout(p=0.5, inplace=False)
  (6): Linear(in_features=4096, out_features=1000, bias=True)
)

In [7]:
# 사전학습모델의 분류기를 현 문제에 맞게 수정
model.classifier = nn.Sequential(
    nn.Linear(512*7*7, 4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096, 4096),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(4096, 10))

model.to(device)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

### 학습 정의 

In [28]:
lr = 0.001
optim = optim.Adam(model.parameters(), lr=lr)

In [31]:
from tqdm import tqdm

epochs = 1

for epoch in range(epochs):
  iterator = tqdm(train_loader)

  for data, label in iterator:
    optim.zero_grad()
    preds = model(data.to(device))
    loss = nn.CrossEntropyLoss()(preds, label.to(device))
    loss.backward()
    optim.step()

    iterator.set_description(f'epoch:{epoch+1} loss:{loss.item()}')

torch.save(model.state_dict(), 'CIFAR_pretrained.pth')

epoch:1 loss:2.307931900024414: 100%|██████████| 1563/1563 [13:19<00:00,  1.95it/s]


### 프리트레인 + 파인튜닝 모델 성능평가 

In [33]:
model.load_state_dict(torch.load('CIFAR_pretrained.pth',
                                 map_location=device))

num_corr = 0

with torch.no_grad():
  for data, label in tqdm(test_loader):

    output = model(data.to(device))
    preds = output.data.max(1)[1]
    corr = preds.eq(label.to(device).data).sum().item()
    num_corr += corr

num_corr / len(test_data)

100%|██████████| 313/313 [01:02<00:00,  5.00it/s]


0.0986